In [1]:
# Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Importing data into pandas
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Dropping columns that won't be used
application_df = application_df.drop(columns=['EIN','NAME'])

# Dropping all rows with a 'STATUS' of 0
application_df = application_df[application_df.STATUS != 0]
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#application_df['APPLICATION_TYPE'].value_counts()
#application_df['CLASSIFICATION'].value_counts()
#application_df['INCOME_AMT'].value_counts()
#application_df['SPECIAL_CONSIDERATIONS'].value_counts()
#application_df['ASK_AMT'].value_counts()

In [4]:
# Creating list of application types to combine, according to our cutoff value of 500+
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Iterating through specified values to replace dataframe values with 'Other'
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Viewing results
application_df['APPLICATION_TYPE'].value_counts()

T3       27032
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Creating list of classifications to combine, according to cutoff of 1800+
classifications_to_replace = ['C7000', 'C1700', 'C4000',
       'C5000', 'C1270', 'C2700', 'C2800', 'C7100', 'C1300', 'C1280', 'C1230',
       'C1400', 'C7200', 'C2300', 'C1240', 'C8000', 'C7120', 'C1500', 'C1800',
       'C6000', 'C1250', 'C8200', 'C1238', 'C1278', 'C1235', 'C1237', 'C7210',
       'C2400', 'C1720', 'C4100', 'C1257', 'C1600', 'C1260', 'C2710', 'C0',
       'C3200', 'C1234', 'C1246', 'C1267', 'C1256', 'C2190', 'C4200', 'C2600',
       'C5200', 'C1370', 'C1248', 'C6100', 'C1820', 'C1900', 'C1236', 'C3700',
       'C2570', 'C1580', 'C1245', 'C2500', 'C1570', 'C1283', 'C2380', 'C1732',
       'C1728', 'C2170', 'C4120', 'C8210', 'C2561', 'C4500', 'C2150']

# Iterating through specified values to replace dataframe values with 'Other'
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Viewing results
application_df['CLASSIFICATION'].value_counts()

C1000    17323
C2000     6073
C1200     4837
Other     2261
C3000     1918
C2100     1882
Name: CLASSIFICATION, dtype: int64

In [6]:
bin00 = application_df[application_df['ASK_AMT'] == 5000]
bin01 = application_df[(application_df['ASK_AMT'] > 5000) & (application_df['ASK_AMT'] <= 100000)]
bin02 = application_df[(application_df['ASK_AMT'] > 100000) & (application_df['ASK_AMT'] <= 1000000)]
bin03 = application_df[(application_df['ASK_AMT'] > 1000000) & (application_df['ASK_AMT'] <= 10000000)]
bin04 = application_df[application_df['ASK_AMT'] > 10000000]

print(bin00.shape, bin01.shape, bin02.shape, bin03.shape, bin04.shape)

(25394, 10) (4369, 10) (2954, 10) (1165, 10) (412, 10)


In [13]:
# Iterating through values to replace dataframe values
for b1 in list(bin01['ASK_AMT']):
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(b1,"5K_100K")

for b2 in list(bin02['ASK_AMT']):
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(b2,'100K_1M')

for b3 in list(bin03['ASK_AMT']):
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(b3,'1M_10M')

for b4 in list(bin04['ASK_AMT']):
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(b4,'over10M')

In [14]:
# Viewing results
application_df['ASK_AMT'].value_counts()

5000       25394
5K_100K     4369
100K_1M     2954
1M_10M      1165
over10M      412
Name: ASK_AMT, dtype: int64

In [15]:
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,100K_1M,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,5K_100K,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,100K_1M,1


In [16]:
# Converting categorical data into numerical data
df = pd.get_dummies(application_df)
df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_5000,ASK_AMT_100K_1M,ASK_AMT_1M_10M,ASK_AMT_5K_100K,ASK_AMT_over10M
0,1,1,0,1,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
1,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,1,0,0,0,0
3,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0


In [17]:
# Splitting data into features and targets
y = df['IS_SUCCESSFUL']
X = df.drop('IS_SUCCESSFUL', axis=1)

# Splitting data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
# Setting up StandardScaler
scaler = StandardScaler()

# Fitting training data
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Defining neural network model
nn = tf.keras.models.Sequential()

In [26]:
# Adding input layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu', input_dim=47))

# Adding hidden layer
nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Adding output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [27]:
# Checking structure of model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                3072      
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 7,297
Trainable params: 7,297
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compiling and training the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

fit_model = nn.fit(X_train_scaled, y_train, epochs=10)

Epoch 1/10
804/804 [==============================] - 1s 892us/step - loss: 0.5717 - accuracy: 0.7219
Epoch 2/10
804/804 [==============================] - 1s 833us/step - loss: 0.5576 - accuracy: 0.7258
Epoch 3/10
804/804 [==============================] - 1s 832us/step - loss: 0.5531 - accuracy: 0.7266
Epoch 4/10
804/804 [==============================] - 1s 827us/step - loss: 0.5510 - accuracy: 0.7285
Epoch 5/10
804/804 [==============================] - 1s 831us/step - loss: 0.5496 - accuracy: 0.7294
Epoch 6/10
804/804 [==============================] - 1s 830us/step - loss: 0.5492 - accuracy: 0.7273
Epoch 7/10
804/804 [==============================] - 1s 828us/step - loss: 0.5469 - accuracy: 0.7306
Epoch 8/10
804/804 [==============================] - 1s 827us/step - loss: 0.5455 - accuracy: 0.7306
Epoch 9/10
804/804 [==============================] - 1s 827us/step - loss: 0.5451 - accuracy: 0.7317
Epoch 10/10
804/804 [==============================] - 1s 828us/step - loss: 0.544

In [29]:
# Evaluating model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5490 - accuracy: 0.7380 - 259ms/epoch - 968us/step
Loss: 0.5490144491195679, Accuracy: 0.7380452752113342


In [ ]:
# Next steps: improve upon the accuracy, try changing other hyperparameters

In [19]:
# Creating function that creates a new sequential model with variable hyperparameters
def create_model(hp):

    nn_optimized = tf.keras.models.Sequential()

    activation = hp.Choice('activation'['relu','tanh','sigmoid'])

    nn_optimized.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=100,
        step=2), activation=activation, input_dim=43))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_optimized.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=2,
            step=2),
            activation=activation))
    
    nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

    nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn_optimized

<>:6: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
<>:6: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
C:\Users\Ray\AppData\Local\Temp/ipykernel_13632/2706221905.py:6: SyntaxWarning: str indices must be integers or slices, not tuple; perhaps you missed a comma?
  activation = hp.Choice('activation'['relu','tanh','sigmoid'])


In [20]:
# Importing keras tuner
import keras_tuner as kt

# Initializing tuner
tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=10,
    hyperband_iterations=2
)

TypeError: string indices must be integers